# Hackathon Example

Das Ziel des Hackathons ist, Energiedaten zu kombinieren und interessante Resultate zu visualisieren. Dazu verwenden wir am besten Jupyter Notebooks wie dieses hier.

In einem Notebook können wir Markdown-Code mit Python kombinieren, die einzelnen Code-Blöcke immer wieder anpassen, und die Resultate direkt modifizieren.

## Tools

### Python
Selbstredend.

### Jupyter Notebook Extension

Extension von Visual Studio Code. [Hier installieren](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter).

### Bibliotheken
####  matplotlib (pyplot)
Eine Python-Bibliothek (also eine Sammlung von Funktionen), um mit Python schöne Grafiken zu erstellen.
#### Pandas
Data-Science Bibliothek
#### Folium
Geo-Visualisierungstools

Generell lassen sich gängige Bibliotheken vom Terminal mit `pip` installieren
```bash
python -m pip install matplotlib pandas folium
```
Wir können alternativ aber auch einfach sicherstellen, dass der _Kernel_, der den Python-Code dieses Notebooks ausführt, die Bibliotheke installiert hat:

In [15]:
import sys
!{sys.executable} -m pip install matplotlib pandas folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 KB 10.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 KB 6.8 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


## Comma-Separated Values

Wir wollen die öffentlich verfügbaren Datensätze verarbeiten. Die tabellarischen Daten liegen oft im CSV-Format (Comma-Separated Values) vor. Dabei repräsentiert jede Zeile ein Eintrag, wobei die Werte der einzelnen Spalten mit Kommas (manchmal auch Semikolons) abgetrennt sind. Enthält ein Wert ein Komma, kann der Wert in Anführungszeichen eingeschlossen sein. Die erste Zeile enthält oft die Spaltennamen:

```csv
Postleitzahl,Ort,Einwohner
8590,Romanshorn,11296
8580,Amriswil,14157
9320,Arbon,14872
```

Wir verwenden gleich die Pandas-Bibliothek, um CSV zu lesen.

Zum Beispiel möchten wir mit den Daten zur Energieproduktion der Thurgauer Gemeinden arbeiten. Das Dataset findet sich unter https://data.tg.ch/explore/dataset/div-energie-10/table/, den Link zum CSV-Download findet sich unter _Exporte_. Der Download verwendet Semikolons.

Die Spaltennamen entnehmen wir ebenfalls der Webseite, um die Gemeinde und deren Energieproduktion zu erhalten.

In [27]:
import pandas as pd
import folium

source = 'https://data.tg.ch/api/explore/v2.1/catalog/datasets/div-energie-10/exports/csv'

# CSV-Daten mit Pandas lesen.
# Achtung: wir verwenden Semikolons statt Kommas als Trennzeichen.
# header=0 teilt der Funktion mit, dass wir Spaltenheader in der Datei erwarten.
energy_dataset = pd.read_table(source, sep=';', header=0)
# Wie sehen die Daten aus?
energy_dataset

,jahr,bfs_nr_gemeinde,gemeinde_name,einwohner,wasserkraft,biomasse_holz,biogasanlagen_landwirtschaft,kehricht,biogasanlagen_industrie,biogasanlagen_abwasser,photovoltaik,wind,total
0,2020,4881,Amlikon-Bissegg,1329,NaN,NaN,3.894,NaN,NaN,NaN,4.366,NaN,8.260
1,2020,4461,Amriswil,14157,NaN,NaN,NaN,NaN,NaN,NaN,5.061,NaN,5.061
2,2020,4891,Berg (TG),3420,NaN,NaN,0.730,NaN,NaN,NaN,2.475,0.005,3.210
3,2020,4901,Birwinken,1330,NaN,NaN,NaN,NaN,NaN,NaN,0.971,NaN,0.971
4,2020,4471,Bischofszell,5878,2.813,NaN,NaN,NaN,NaN,1.336,2.102,NaN,6.251
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2015,4871,Wagenhausen,1672,NaN,NaN,NaN,NaN,NaN,NaN,0.259,NaN,0.259
476,2015,4946,Weinfelden,11147,8.266,NaN,NaN,39.873,NaN,0.474,2.011,NaN,50.624
477,2015,4951,Wigoltingen,2369,NaN,NaN,0.398,NaN,NaN,NaN,1.254,NaN,1.652
478,2015,4791,Wuppenau,1120,NaN,NaN,NaN,NaN,NaN,NaN,0.720,NaN,0.720


Für den Anfang interessieren uns nur die Werte 2020. Zudem möchten wir nur die Gemeinde und die totale Energieproduktion.

In [34]:
only2020 = energy_dataset[energy_dataset['jahr'] == 2020]
only2020[['gemeinde_name', 'total']]

,gemeinde_name,total
0,Amlikon-Bissegg,8.260
1,Amriswil,5.061
2,Berg (TG),3.210
3,Birwinken,0.971
4,Bischofszell,6.251
...,...,...
75,Wäldi,1.552
76,Weinfelden,73.173
77,Wigoltingen,3.122
78,Wuppenau,1.271


## Daten zusammenfassen und Visualisieren

TBD